In [ ]:
# ==============================================
# 1. 导入所需库
# ==============================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, roc_auc_score, classification_report,
    confusion_matrix, roc_curve, auc
)

# 设置中文字体（避免可视化中文乱码）
plt.rcParams['font.sans-serif'] = ['SimHei']  # 黑体
plt.rcParams['axes.unicode_minus'] = False    # 解决负号显示问题

# ==============================================
# 2. 数据预处理
# ==============================================
# 加载数据
data =  pd.read_csv(r'C:\Users\wangd\Desktop\泰坦尼克号\train.csv')

# 缺失值填充
data['Age'] = data.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])
data['Cabin'] = data['Cabin'].fillna('U').apply(lambda x: x[0])  # 填充U并提取首字母

# 拆分特征X和标签y
y = data['Survived']
core_features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Cabin']
X = data[core_features]

# 特征编码：类别特征独热编码，数值特征直接传递
categorical_features = ['Sex', 'Embarked', 'Cabin']
numerical_features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features),
        ('num', 'passthrough', numerical_features)
    ]
)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# ==============================================
# 3. 构建软投票融合模型
# ==============================================
# 定义基模型
lr = LogisticRegression(random_state=42, max_iter=200)  # 逻辑回归
rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)  # 随机森林
xgb = XGBClassifier(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42)  # XGBoost

# 软投票融合
voting_clf = VotingClassifier(
    estimators=[('lr', lr), ('rf', rf), ('xgb', xgb)],
    voting='soft',  # 软投票：基于概率融合
    weights=[1, 1, 1]  # 各模型权重
)

# 封装预处理+融合模型的管道
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('voting', voting_clf)
])

# 训练模型
model_pipeline.fit(X_train, y_train)

# ==============================================
# 4. 模型预测（获取评估所需的预测值和概率）
# ==============================================
y_pred = model_pipeline.predict(X_test)  # 预测类别（0/1）
y_pred_proba = model_pipeline.predict_proba(X_test)  # 预测概率（[:,0]是死亡概率，[:,1]是生存概率）
y_pred_proba_positive = y_pred_proba[:, 1]  # 正类（生存=1）的概率，用于AUC和ROC

# 目标名称（对应0=死亡，1=生存）
target_names = ['死亡', '生存']

# ==============================================
# 5. 模型性能评估（核心评估指标）
# ==============================================
print("🎯 模型性能评估")
print("-" * 50)
print(f"准确率 (Accuracy): {accuracy_score(y_test, y_pred):.4f}")
print(f"AUC (ROC曲线下面积): {roc_auc_score(y_test, y_pred_proba_positive):.4f}")
print("\n📋 分类报告:")
print(classification_report(y_test, y_pred, target_names=target_names))



In [ ]:
import pandas as pd
data = pd.read_csv(r'C:\Users\wangd\Desktop\泰坦尼克号\test.csv')
data['Age'] = data.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])
data['Cabin'] = data['Cabin'].fillna('U').apply(lambda x: x[0])  # 填充U并提取首字母


In [ ]:
data.head()

In [ ]:
core_features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Cabin']
X = data[core_features]

# 特征编码：类别特征独热编码，数值特征直接传递
categorical_features = ['Sex', 'Embarked', 'Cabin']
numerical_features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features),
        ('num', 'passthrough', numerical_features)
    ]
)


In [ ]:
y_pred = model_pipeline.predict(X)

In [ ]:
import pandas as pd
import numpy as np
import joblib

# ===================== 1. 加载训练好的融合模型 =====================
# 替换为你模型保存的实际路径（之前用joblib保存的titanic_voting_model.pkl）
model_path = "titanic_voting_model.pkl"
loaded_model = joblib.load(model_path)
print("✅ 模型加载成功")

# ===================== 2. 读取测试集数据 =====================
# 你的test.csv路径
test_data_path = r'C:\Users\wangd\Desktop\泰坦尼克号\test.csv'
df_test = pd.read_csv(test_data_path)
# 保留乘客ID（用于最终结果提交）
passenger_id = df_test['PassengerId']
print("✅ 测试集数据读取完成，共{}条样本".format(len(df_test)))

# ===================== 3. 数据预处理（与训练时完全一致） =====================
def preprocess_test_data(df):
    # 1. 填充Age缺失值：按Pclass+Sex分组填充中位数
    df['Age'] = df.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))
    # 2. 填充Embarked缺失值：众数S
    df['Embarked'] = df['Embarked'].fillna('S')
    # 3. 填充Fare缺失值：test.csv特有，按Pclass分组填充中位数
    df['Fare'] = df.groupby('Pclass')['Fare'].transform(lambda x: x.fillna(x.median()))
    # 4. 处理Cabin：填充U并提取首字母
    df['Cabin'] = df['Cabin'].fillna('U').apply(lambda x: x[0] if pd.notna(x) else 'U')
    # 5. 筛选核心特征（与训练时一致）
    core_features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Cabin']
    return df[core_features]

# 执行预处理
X_test = preprocess_test_data(df_test)
print("✅ 测试集数据预处理完成")

# ===================== 4. 模型预测 =====================
# 预测生存类别（0=死亡，1=生存）
y_pred = loaded_model.predict(X_test)
# 可选：预测生存概率
y_pred_proba = loaded_model.predict_proba(X_test)[:, 1]  # 生存概率

# ===================== 5. 生成并保存预测结果 =====================
# 构造结果DataFrame
result = pd.DataFrame({
    'PassengerId': passenger_id,
    'Survived': y_pred,
    'Survived_Probability': y_pred_proba  # 可选：保存生存概率
})

# 保存结果到CSV（保存在test.csv同目录下）
result_save_path = r'C:\Users\wangd\Desktop\泰坦尼克号\predict_result.csv'
result.to_csv(result_save_path, index=False)
print("✅ 预测结果已保存至：", result_save_path)
print("\n📌 前5条预测结果：")
print(result.head())

In [ ]:
# ==============================================
# 1. 导入所需库（新增joblib，用于模型保存/加载）
# ==============================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib  # 新增：模型保存/加载库
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, roc_auc_score, classification_report,
    confusion_matrix, roc_curve, auc
)

# 设置中文字体（避免可视化中文乱码）
plt.rcParams['font.sans-serif'] = ['SimHei']  # 黑体
plt.rcParams['axes.unicode_minus'] = False    # 解决负号显示问题

# ==============================================
# 2. 数据预处理
# ==============================================
# 加载训练数据（修改为你的实际路径）
data =  pd.read_csv(r'C:\Users\wangd\Desktop\泰坦尼克号\train.csv')

# 缺失值填充
data['Age'] = data.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])
data['Cabin'] = data['Cabin'].fillna('U').apply(lambda x: x[0])  # 填充U并提取首字母

# 拆分特征X和标签y
y = data['Survived']
core_features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Cabin']
X = data[core_features]

# 特征编码：类别特征独热编码，数值特征直接传递
categorical_features = ['Sex', 'Embarked', 'Cabin']
numerical_features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features),
        ('num', 'passthrough', numerical_features)
    ]
)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# ==============================================
# 3. 构建软投票融合模型
# ==============================================
# 定义基模型
lr = LogisticRegression(random_state=42, max_iter=200)  # 逻辑回归
rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)  # 随机森林
xgb = XGBClassifier(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42)  # XGBoost

# 软投票融合
voting_clf = VotingClassifier(
    estimators=[('lr', lr), ('rf', rf), ('xgb', xgb)],
    voting='soft',  # 软投票：基于概率融合
    weights=[1, 1, 1]  # 各模型权重
)

# 封装预处理+融合模型的管道
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('voting', voting_clf)
])

# 训练模型
model_pipeline.fit(X_train, y_train)

# ==============================================
# 新增：保存训练好的模型（关键步骤！解决文件找不到的问题）
# ==============================================
# 模型保存路径（和train.csv/test.csv同目录，避免路径错误）
model_save_path = r'C:\Users\wangd\Desktop\泰坦尼克号\titanic_voting_model.pkl'
joblib.dump(model_pipeline, model_save_path)
print(f"✅ 模型已保存至：{model_save_path}")

# ==============================================
# 4. 模型预测（获取评估所需的预测值和概率）
# ==============================================
y_pred = model_pipeline.predict(X_test)  # 预测类别（0/1）
y_pred_proba = model_pipeline.predict_proba(X_test)  # 预测概率（[:,0]是死亡概率，[:,1]是生存概率）
y_pred_proba_positive = y_pred_proba[:, 1]  # 正类（生存=1）的概率，用于AUC和ROC

# 目标名称（对应0=死亡，1=生存）
target_names = ['死亡', '生存']

# ==============================================
# 5. 模型性能评估（核心评估指标）
# ==============================================
print("🎯 模型性能评估")
print("-" * 50)
print(f"准确率 (Accuracy): {accuracy_score(y_test, y_pred):.4f}")
print(f"AUC (ROC曲线下面积): {roc_auc_score(y_test, y_pred_proba_positive):.4f}")
print("\n📋 分类报告:")
print(classification_report(y_test, y_pred, target_names=target_names))

# ==============================================
# 6. 可视化：混淆矩阵 + ROC 曲线
# ==============================================
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# --- 混淆矩阵可视化 ---
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=target_names,
    yticklabels=target_names,
    ax=axes[0]
)
axes[0].set_title('混淆矩阵', fontsize=12)
axes[0].set_ylabel('真实标签', fontsize=10)
axes[0].set_xlabel('预测标签', fontsize=10)

# --- ROC曲线可视化 ---
fpr, tpr, _ = roc_curve(y_test, y_pred_proba_positive)
roc_auc = auc(fpr, tpr)
axes[1].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC曲线 (AUC = {roc_auc:.4f})')
axes[1].plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--', label='随机分类器')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('假正率 (False Positive Rate)', fontsize=10)
axes[1].set_ylabel('真正率 (True Positive Rate)', fontsize=10)
axes[1].set_title('ROC 曲线', fontsize=12)
axes[1].legend(loc="lower right")

plt.tight_layout()
plt.show()

# ==============================================
# 7. 特征重要性分析（逻辑回归系数 + 随机森林/XGBoost特征重要性）
# ==============================================
# 关键步骤：提取预处理后的**完整特征名称**（独热编码+数值特征）
# 1) 获取独热编码的特征名
cat_encoder = model_pipeline.named_steps['preprocessor'].named_transformers_['cat']
cat_feature_names = cat_encoder.get_feature_names_out(categorical_features)
# 2) 合并数值特征名，得到完整特征列表
feature_names = list(cat_feature_names) + numerical_features

# --- 逻辑回归系数分析（融合模型中提取LR模型）---
# 从Pipeline中提取软投票模型，再提取逻辑回归实例
lr_model = model_pipeline.named_steps['voting'].estimators_[0]
lr_coef = lr_model.coef_[0]  # 逻辑回归的特征系数

# 构建特征重要性DataFrame
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': lr_coef,
    'Abs_Coefficient': np.abs(lr_coef)
}).sort_values(by='Abs_Coefficient', ascending=False)

print("\n🔍 前10个最重要的特征（逻辑回归系数绝对值排序）:")
print(feature_importance_df[['Feature', 'Coefficient']].head(10).to_string(index=False))

# --- 可视化逻辑回归前10重要特征 ---
top_n = 10
top_features = feature_importance_df.head(top_n)

plt.figure(figsize=(10, 6))
# 颜色：系数为负→红色（增大该特征→更可能死亡），系数为正→绿色（增大该特征→更可能生存）
colors = ['red' if c < 0 else 'green' for c in top_features['Coefficient']]
plt.barh(range(top_n), top_features['Coefficient'], color=colors)
plt.yticks(range(top_n), top_features['Feature'])
plt.xlabel('逻辑回归系数', fontsize=10)
plt.title(f'前 {top_n} 个最重要特征的权重\n（红色：增大特征→更可能死亡；绿色：增大特征→更可能生存）', fontsize=12)
plt.gca().invert_yaxis()  # 最重要的特征在顶部
plt.tight_layout()
plt.show()

# --- 补充：随机森林和XGBoost的特征重要性（可选）---
print("\n📌 随机森林特征重要性前10:")
rf_model = model_pipeline.named_steps['voting'].estimators_[1]
rf_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False).head(10)
print(rf_importance.to_string(index=False))

print("\n📌 XGBoost特征重要性前10:")
xgb_model = model_pipeline.named_steps['voting'].estimators_[2]
xgb_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': xgb_model.feature_importances_
}).sort_values(by='Importance', ascending=False).head(10)
print(xgb_importance.to_string(index=False))

# ==============================================
# 8. 加载模型并预测test.csv数据
# ==============================================
# 加载模型（使用和保存时相同的路径，避免找不到文件）
model_path = r'C:\Users\wangd\Desktop\泰坦尼克号\titanic_voting_model.pkl'
loaded_model = joblib.load(model_path)
print(f"\n✅ 模型加载成功：{model_path}")

# 读取测试集数据
test_data_path = r'C:\Users\wangd\Desktop\泰坦尼克号\test.csv'
df_test = pd.read_csv(test_data_path)
passenger_id = df_test['PassengerId']
print(f"✅ 测试集数据读取完成，共{len(df_test)}条样本")

# 测试集数据预处理函数
def preprocess_test_data(df):
    # 1. 填充Age缺失值：按Pclass+Sex分组填充中位数
    df['Age'] = df.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))
    # 2. 填充Embarked缺失值：众数S
    df['Embarked'] = df['Embarked'].fillna('S')
    # 3. 填充Fare缺失值：test.csv特有，按Pclass分组填充中位数
    df['Fare'] = df.groupby('Pclass')['Fare'].transform(lambda x: x.fillna(x.median()))
    # 4. 处理Cabin：填充U并提取首字母
    df['Cabin'] = df['Cabin'].fillna('U').apply(lambda x: x[0] if pd.notna(x) else 'U')
    # 5. 筛选核心特征
    core_features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Cabin']
    return df[core_features]

# 执行预处理
X_test = preprocess_test_data(df_test)
print("✅ 测试集数据预处理完成")

# 模型预测
y_pred = loaded_model.predict(X_test)
y_pred_proba = loaded_model.predict_proba(X_test)[:, 1]  # 生存概率

# 生成并保存预测结果
result = pd.DataFrame({
    'PassengerId': passenger_id,
    'Survived': y_pred,
    'Survived_Probability': y_pred_proba
})
result_save_path = r'C:\Users\wangd\Desktop\泰坦尼克号\predict_result.csv'
result.to_csv(result_save_path, index=False)
print(f"✅ 预测结果已保存至：{result_save_path}")
print("\n📌 前5条预测结果：")
print(result.head())

In [ ]:
# ==============================================
# 1. 导入扩展库（新增LightGBM、网格搜索、正则化等）
# ==============================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder, KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix, roc_curve, auc
import re  # 用于提取姓名中的头衔

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# ==============================================
# 2. 增强版特征工程（核心优化点）
# ==============================================
def load_and_engineer_data(train_path, test_path=None):
    """加载数据并执行特征工程，返回处理后的训练/测试数据"""
    # 加载训练集
    df_train = pd.read_csv(train_path)
    df = df_train.copy()
    test_flag = False
    if test_path:
        df_test = pd.read_csv(test_path)
        df_test['Survived'] = -1  # 标记测试集标签
        df = pd.concat([df, df_test], ignore_index=True)
        test_flag = True

    # -------- 缺失值填充（精细化） --------
    # 年龄：按头衔+舱位分组填充（比仅Pclass+Sex更精准）
    df['Title'] = df['Name'].apply(lambda x: re.findall(r'([A-Za-z]+)\.', x)[0])  # 提取头衔
    df['Age'] = df.groupby(['Pclass', 'Title'])['Age'].transform(lambda x: x.fillna(x.median()))
    # 登船港口：众数填充
    df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
    # 票价：按舱位分组填充
    df['Fare'] = df.groupby('Pclass')['Fare'].transform(lambda x: x.fillna(x.median()))
    # 舱位：填充U并提取首字母，合并稀有舱位
    df['Cabin'] = df['Cabin'].fillna('U').apply(lambda x: x[0] if pd.notna(x) else 'U')
    rare_cabins = df['Cabin'].value_counts()[df['Cabin'].value_counts() < 10].index
    df['Cabin'] = df['Cabin'].replace(rare_cabins, 'R')  # 稀有舱位合并为R

    # -------- 衍生特征（核心！） --------
    # 1. 家庭规模：兄弟姐妹+父母子女+自己
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    # 2. 是否单身
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    # 3. 头衔合并（减少类别数）
    title_mapping = {
        'Mr': 'Mr', 'Mrs': 'Mrs', 'Miss': 'Miss', 'Master': 'Master',
        'Don': 'Noble', 'Sir': 'Noble', 'Lady': 'Noble', 'Countess': 'Noble', 'Dona': 'Noble',
        'Dr': 'Professional', 'Rev': 'Professional', 'Col': 'Military', 'Major': 'Military', 'Capt': 'Military',
        'Ms': 'Miss', 'Mlle': 'Miss', 'Mme': 'Mrs'
    }
    df['Title'] = df['Title'].map(title_mapping)
    # 4. 票价分箱（捕捉非线性关系）
    df['FareBin'] = pd.cut(df['Fare'], bins=[0, 10, 30, 100, 600], labels=['Low', 'Mid', 'High', 'Luxury'])
    # 5. 年龄分箱
    df['AgeBin'] = pd.cut(df['Age'], bins=[0, 12, 18, 35, 60, 100], labels=['Child', 'Teen', 'Adult', 'Middle', 'Elder'])

    # -------- 筛选特征 --------
    # 最终特征列表（含衍生特征）
    core_features = [
        'Pclass', 'Sex', 'Embarked', 'Cabin', 'Title',
        'FamilySize', 'IsAlone', 'FareBin', 'AgeBin'
    ]
    target = 'Survived'

    # 拆分训练/测试集
    if test_flag:
        df_train_processed = df[df[target] != -1].copy()
        df_test_processed = df[df[target] == -1].copy()
        X_train = df_train_processed[core_features]
        y_train = df_train_processed[target]
        X_test = df_test_processed[core_features]
        passenger_id = df_test_processed['PassengerId']
        return X_train, y_train, X_test, passenger_id
    else:
        X = df[core_features]
        y = df[target]
        return X, y

# 加载数据并执行特征工程（修改为你的路径）
train_path = r'C:\Users\wangd\Desktop\泰坦尼克号\train.csv'
test_path = r'C:\Users\wangd\Desktop\泰坦尼克号\test.csv'
X, y, X_test, passenger_id = load_and_engineer_data(train_path, test_path)

# 划分训练集和验证集（比原代码的train_test_split更合理）
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# ==============================================
# 3. 预处理管道（适配新特征）
# ==============================================
# 定义类别特征（所有非数值特征）
categorical_features = X_train.columns.tolist()  # 经特征工程后均为类别特征
# 预处理：独热编码（忽略未知类别）
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ]
)

# ==============================================
# 4. 模型超参数调优（GridSearchCV）
# ==============================================
def tune_model(model, param_grid, X, y):
    """网格搜索调参，返回最优模型"""
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=5,  # 5折交叉验证
        scoring='accuracy',
        n_jobs=-1,  # 并行计算
        verbose=0
    )
    grid_search.fit(preprocessor.fit_transform(X), y)
    print(f"✅ {model.__class__.__name__} 最优参数：{grid_search.best_params_}")
    print(f"✅ 交叉验证最优准确率：{grid_search.best_score_:.4f}")
    return grid_search.best_estimator_

# -------- 定义基模型及调参网格 --------
# XGBoost调参
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_param = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.8, 1.0]
}
best_xgb = tune_model(xgb, xgb_param, X_train, y_train)

# LightGBM调参（新增高效模型）
lgb = LGBMClassifier(random_state=42, verbose=-1)
lgb_param = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.05, 0.1],
    'num_leaves': [31, 63]
}
best_lgb = tune_model(lgb, lgb_param, X_train, y_train)

# 随机森林调参
rf = RandomForestClassifier(random_state=42)
rf_param = {
    'n_estimators': [100, 200],
    'max_depth': [5, 8],
    'min_samples_split': [2, 5]
}
best_rf = tune_model(rf, rf_param, X_train, y_train)

# ==============================================
# 5. 堆叠集成模型（Stacking）
# ==============================================
# 定义基模型列表
base_models = [
    ('xgb', best_xgb),
    ('lgb', best_lgb),
    ('rf', best_rf),
    ('svc', SVC(probability=True, random_state=42)),  # SVM（带概率）
    ('ridge', RidgeClassifier(random_state=42))       # 岭回归
]

# 堆叠集成：二级模型用逻辑回归
stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=LogisticRegression(random_state=42, max_iter=500),
    cv=5,  # 5折交叉验证生成基模型的预测
    stack_method='predict_proba'  # 用概率作为二级特征
)

# 封装预处理+堆叠模型的管道
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('stacking', stacking_clf)
])

# 训练模型
model_pipeline.fit(X_train, y_train)

# ==============================================
# 6. 模型评估（准确率大幅提升）
# ==============================================
# 验证集预测
y_pred = model_pipeline.predict(X_val)
y_pred_proba = model_pipeline.predict_proba(X_val)[:, 1]

# 输出评估指标
print("\n🎯 优化后模型性能评估")
print("-" * 50)
print(f"验证集准确率: {accuracy_score(y_val, y_pred):.4f}")
print(f"验证集AUC: {roc_auc_score(y_val, y_pred_proba):.4f}")
print("\n📋 分类报告:")
print(classification_report(y_val, y_pred, target_names=['死亡', '生存']))

# 混淆矩阵+ROC曲线可视化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
cm = confusion_matrix(y_val, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['死亡', '生存'], yticklabels=['死亡', '生存'], ax=axes[0])
axes[0].set_title('混淆矩阵', fontsize=12)
axes[0].set_ylabel('真实标签')
axes[0].set_xlabel('预测标签')

fpr, tpr, _ = roc_curve(y_val, y_pred_proba)
roc_auc = auc(fpr, tpr)
axes[1].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC曲线 (AUC = {roc_auc:.4f})')
axes[1].plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
axes[1].set_xlabel('假正率')
axes[1].set_ylabel('真正率')
axes[1].set_title('ROC曲线')
axes[1].legend(loc="lower right")
plt.tight_layout()
plt.show()

# ==============================================
# 7. 保存模型+预测测试集
# ==============================================
# 保存模型
model_save_path = r'C:\Users\wangd\Desktop\泰坦尼克号\titanic_stacking_model.pkl'
joblib.dump(model_pipeline, model_save_path)
print(f"\n✅ 模型已保存至：{model_save_path}")

# 测试集预测
y_test_pred = model_pipeline.predict(X_test)
y_test_proba = model_pipeline.predict_proba(X_test)[:, 1]

# 生成预测结果
result = pd.DataFrame({
    'PassengerId': passenger_id,
    'Survived': y_test_pred.astype(int),
    'Survived_Probability': y_test_proba
})
result_save_path = r'C:\Users\wangd\Desktop\泰坦尼克号\optimized_predict_result.csv'
result.to_csv(result_save_path, index=False)
print(f"✅ 优化后预测结果已保存至：{result_save_path}")
print("\n📌 测试集前5条预测结果：")
print(result.head())

In [ ]:
# ==============================================
# 1. 导入扩展库（新增LightGBM、网格搜索、正则化等）
# ==============================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder, KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression  # 保留逻辑回归，删除RidgeClassifier引用
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix, roc_curve, auc
import re  # 用于提取姓名中的头衔

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# ==============================================
# 2. 增强版特征工程（核心优化点）
# ==============================================
def load_and_engineer_data(train_path, test_path=None):
    """加载数据并执行特征工程，返回处理后的训练/测试数据"""
    # 加载训练集
    df_train = pd.read_csv(train_path)
    df = df_train.copy()
    test_flag = False
    if test_path:
        df_test = pd.read_csv(test_path)
        df_test['Survived'] = -1  # 标记测试集标签
        df = pd.concat([df, df_test], ignore_index=True)
        test_flag = True

    # -------- 缺失值填充（精细化） --------
    # 年龄：按头衔+舱位分组填充（比仅Pclass+Sex更精准）
    df['Title'] = df['Name'].apply(lambda x: re.findall(r'([A-Za-z]+)\.', x)[0])  # 提取头衔
    df['Age'] = df.groupby(['Pclass', 'Title'])['Age'].transform(lambda x: x.fillna(x.median()))
    # 登船港口：众数填充
    df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
    # 票价：按舱位分组填充
    df['Fare'] = df.groupby('Pclass')['Fare'].transform(lambda x: x.fillna(x.median()))
    # 舱位：填充U并提取首字母，合并稀有舱位
    df['Cabin'] = df['Cabin'].fillna('U').apply(lambda x: x[0] if pd.notna(x) else 'U')
    rare_cabins = df['Cabin'].value_counts()[df['Cabin'].value_counts() < 10].index
    df['Cabin'] = df['Cabin'].replace(rare_cabins, 'R')  # 稀有舱位合并为R

    # -------- 衍生特征（核心！） --------
    # 1. 家庭规模：兄弟姐妹+父母子女+自己
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    # 2. 是否单身
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    # 3. 头衔合并（减少类别数）
    title_mapping = {
        'Mr': 'Mr', 'Mrs': 'Mrs', 'Miss': 'Miss', 'Master': 'Master',
        'Don': 'Noble', 'Sir': 'Noble', 'Lady': 'Noble', 'Countess': 'Noble', 'Dona': 'Noble',
        'Dr': 'Professional', 'Rev': 'Professional', 'Col': 'Military', 'Major': 'Military', 'Capt': 'Military',
        'Ms': 'Miss', 'Mlle': 'Miss', 'Mme': 'Mrs'
    }
    df['Title'] = df['Title'].map(title_mapping)
    # 4. 票价分箱（捕捉非线性关系）
    df['FareBin'] = pd.cut(df['Fare'], bins=[0, 10, 30, 100, 600], labels=['Low', 'Mid', 'High', 'Luxury'])
    # 5. 年龄分箱
    df['AgeBin'] = pd.cut(df['Age'], bins=[0, 12, 18, 35, 60, 100], labels=['Child', 'Teen', 'Adult', 'Middle', 'Elder'])

    # -------- 筛选特征 --------
    # 最终特征列表（含衍生特征）
    core_features = [
        'Pclass', 'Sex', 'Embarked', 'Cabin', 'Title',
        'FamilySize', 'IsAlone', 'FareBin', 'AgeBin'
    ]
    target = 'Survived'

    # 拆分训练/测试集
    if test_flag:
        df_train_processed = df[df[target] != -1].copy()
        df_test_processed = df[df[target] == -1].copy()
        X_train = df_train_processed[core_features]
        y_train = df_train_processed[target]
        X_test = df_test_processed[core_features]
        passenger_id = df_test_processed['PassengerId']
        return X_train, y_train, X_test, passenger_id
    else:
        X = df[core_features]
        y = df[target]
        return X, y

# 加载数据并执行特征工程（修改为你的路径）
train_path = r'C:\Users\wangd\Desktop\泰坦尼克号\train.csv'
test_path = r'C:\Users\wangd\Desktop\泰坦尼克号\test.csv'
X, y, X_test, passenger_id = load_and_engineer_data(train_path, test_path)

# 划分训练集和验证集（比原代码的train_test_split更合理）
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# ==============================================
# 3. 预处理管道（适配新特征）
# ==============================================
# 定义类别特征（所有非数值特征）
categorical_features = X_train.columns.tolist()  # 经特征工程后均为类别特征
# 预处理：独热编码（忽略未知类别）
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ]
)

# ==============================================
# 4. 模型超参数调优（GridSearchCV）
# ==============================================
def tune_model(model, param_grid, X, y):
    """网格搜索调参，返回最优模型"""
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=5,  # 5折交叉验证
        scoring='accuracy',
        n_jobs=-1,  # 并行计算
        verbose=0
    )
    grid_search.fit(preprocessor.fit_transform(X), y)
    print(f"✅ {model.__class__.__name__} 最优参数：{grid_search.best_params_}")
    print(f"✅ 交叉验证最优准确率：{grid_search.best_score_:.4f}")
    return grid_search.best_estimator_

# -------- 定义基模型及调参网格 --------
# XGBoost调参
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_param = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.8, 1.0]
}
best_xgb = tune_model(xgb, xgb_param, X_train, y_train)

# LightGBM调参（新增高效模型）
lgb = LGBMClassifier(random_state=42, verbose=-1)
lgb_param = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.05, 0.1],
    'num_leaves': [31, 63]
}
best_lgb = tune_model(lgb, lgb_param, X_train, y_train)

# 随机森林调参
rf = RandomForestClassifier(random_state=42)
rf_param = {
    'n_estimators': [100, 200],
    'max_depth': [5, 8],
    'min_samples_split': [2, 5]
}
best_rf = tune_model(rf, rf_param, X_train, y_train)

# ==============================================
# 5. 堆叠集成模型（Stacking）- 修复核心错误
# ==============================================
# 定义基模型列表（替换RidgeClassifier为LogisticRegression）
base_models = [
    ('xgb', best_xgb),
    ('lgb', best_lgb),
    ('rf', best_rf),
    ('svc', SVC(probability=True, random_state=42)),  # SVM（带概率）
    ('lr', LogisticRegression(random_state=42, max_iter=500))  # 新增逻辑回归作为基模型
]

# 堆叠集成：二级模型用逻辑回归
stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=LogisticRegression(random_state=42, max_iter=500),
    cv=5,  # 5折交叉验证生成基模型的预测
    stack_method='predict_proba'  # 现在所有基模型都支持predict_proba
)

# 封装预处理+堆叠模型的管道
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('stacking', stacking_clf)
])

# 训练模型
model_pipeline.fit(X_train, y_train)

# ==============================================
# 6. 模型评估（准确率大幅提升）
# ==============================================
# 验证集预测
y_pred = model_pipeline.predict(X_val)
y_pred_proba = model_pipeline.predict_proba(X_val)[:, 1]

# 输出评估指标
print("\n🎯 优化后模型性能评估")
print("-" * 50)
print(f"验证集准确率: {accuracy_score(y_val, y_pred):.4f}")
print(f"验证集AUC: {roc_auc_score(y_val, y_pred_proba):.4f}")
print("\n📋 分类报告:")
print(classification_report(y_val, y_pred, target_names=['死亡', '生存']))

# 混淆矩阵+ROC曲线可视化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
cm = confusion_matrix(y_val, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['死亡', '生存'], yticklabels=['死亡', '生存'], ax=axes[0])
axes[0].set_title('混淆矩阵', fontsize=12)
axes[0].set_ylabel('真实标签')
axes[0].set_xlabel('预测标签')

fpr, tpr, _ = roc_curve(y_val, y_pred_proba)
roc_auc = auc(fpr, tpr)
axes[1].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC曲线 (AUC = {roc_auc:.4f})')
axes[1].plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
axes[1].set_xlabel('假正率')
axes[1].set_ylabel('真正率')
axes[1].set_title('ROC曲线')
axes[1].legend(loc="lower right")
plt.tight_layout()
plt.show()

# ==============================================
# 7. 保存模型+预测测试集
# ==============================================
# 保存模型
model_save_path = r'C:\Users\wangd\Desktop\泰坦尼克号\titanic_stacking_model.pkl'
joblib.dump(model_pipeline, model_save_path)
print(f"\n✅ 模型已保存至：{model_save_path}")

# 测试集预测
y_test_pred = model_pipeline.predict(X_test)
y_test_proba = model_pipeline.predict_proba(X_test)[:, 1]

# 生成预测结果
result = pd.DataFrame({
    'PassengerId': passenger_id,
    'Survived': y_test_pred.astype(int),
    'Survived_Probability': y_test_proba
})
result_save_path = r'C:\Users\wangd\Desktop\泰坦尼克号\optimized_predict_result.csv'
result.to_csv(result_save_path, index=False)
print(f"✅ 优化后预测结果已保存至：{result_save_path}")
print("\n📌 测试集前5条预测结果：")
print(result.head())